# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096895


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/34 [00:00<?, ?it/s]

Rendering models:   6%|▌         | 2/34 [00:06<01:50,  3.44s/it]

Rendering models:   9%|▉         | 3/34 [00:06<01:15,  2.44s/it]

Rendering models:  12%|█▏        | 4/34 [00:07<00:53,  1.80s/it]

Rendering models:  15%|█▍        | 5/34 [00:07<00:37,  1.31s/it]

...

Rendering models:  91%|█████████ | 31/34 [00:10<00:00,  6.41it/s]

Rendering models:  94%|█████████▍| 32/34 [00:11<00:00,  6.04it/s]

Rendering models:  97%|█████████▋| 33/34 [00:11<00:00,  6.16it/s]

Rendering models: 100%|██████████| 34/34 [00:11<00:00,  6.62it/s]

not-logged-in-0c68db93566efd3108c4    0.001855
not-logged-in-7edd9b2ab9752961abc1    0.030128
jnarayanbvg                           0.000326
c196948                               0.003238
awright5                              0.005036
not-logged-in-c6917d9b19f41e6c2995    0.002137
TSM_Ginga                             0.000805
ElisabethB                            0.066872
not-logged-in-c7fa0939ae81c509c79f    0.001853
not-logged-in-3b165c705d7744fc4c51    0.001056
not-logged-in-7c0e3b1ae8dc7fbb6b66    0.003885
maybeadayoff                          0.000503
pangeli5                              0.000434
pangeli5                              0.000403
AlexYoung35                           0.000601
XxBABYGRUFFALOxX                      0.002071
clairedeu                             0.000911
tominas                               0.006520
tominas                               0.000478
not-logged-in-300dca7f45d1a5d1dd58    0.032148
alexmnelson18                         0.000382
Mr_DMan      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())